# GOAL


MAKE A LLM BUILD A WEBSCRAPER BASED ON THE PROVIDED HTML CONTENT OF SOME PAGES

In [4]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.vectorstores import Chroma 
from langchain.chains import ChatVectorDBChain
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from bs4 import BeautifulSoup
import requests

In [5]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.66 Safari/537.36"
}
url = 'https://www.vivareal.com.br/venda/brasil/casa_residencial/#banheiros=3&preco-ate=500000'

In [7]:
response = requests.get(url, headers=header)
response.raise_for_status()  # Raise an exception if the request was unsuccessful
soup = BeautifulSoup(response.text, 'html.parser')

In [33]:
soup.contents[-2].contents[1].contents[39:46]

[<style type="text/css"> @-webkit-keyframes spinning{0%{transform:rotate(0deg)}to{transform:rotate(1turn)}}@font-face{font-display:swap;font-family:Open Sans;font-style:normal;font-weight:400;src:url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_400.woff2) format("woff2"),url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_400.woff) format("woff"),url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_400.ttf) format("truetype")}@font-face{font-display:swap;font-family:Open Sans;font-style:normal;font-weight:600;src:url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_600.woff2) format("woff2"),url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_600.woff) format("woff"),url(https://cdn1.vivareal.com/p/1-7b1d2fb/v/static/app/fonts/Open_Sans_600.ttf) format("truetype")}a,address,article,aside,b,blockquote,body,button,code,dd,div,dl,dt,em,fieldset,figure,footer,form,h1,h2,h3,h4,h5,h6,header,html,i

In [40]:
soup.contents[-2].contents[3].contents

[' ',
 <nav class="js-site-menu" id="js-site-menu"> <div class="site-menu__wrapper js-site-menu-wrapper"> <div class="site-menu js-site-menu-panel"> <header class="site-menu__header"> <button class="site-menu__close js-site-menu-close"> Fechar </button> </header> <ul class="site-menu__list"> <li> <button class="site-header__login site-header__login--mobile js-login" title="Entrar"> Entrar </button> </li> <li> <a class="js-track" href="/venda/?itl_id=1000177&amp;itl_name=vivareal_-_link-header_comprar_to_vivareal_resultado-pesquisa"> Comprar </a> </li> <li> <a class="js-track" href="/aluguel/?itl_id=1000178&amp;itl_name=vivareal_-_link-header_alugar_to_vivareal_resultado-pesquisa"> Alugar </a> </li> <li> <a class="js-track" href="/imoveis-lancamento/?itl_id=1000179&amp;itl_name=vivareal_-_link-header_lancamentos_to_vivareal_resultado-pesquisa"> Imóveis Novos </a> </li> <li> <a class="js-track" href="/feirao/festival-da-mudanca/?itl_id=1000274&amp;itl_name=vivareal_resultado-pesquisa_lin

In [28]:
count = 0
for i in soup.contents[-2].contents[1].contents:
    print(i, count)
    count += 1

  0
<meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/> 1
  2
<meta content="omni2c019e0" name="omniverify"/> 3
  4
<meta content="https://www.vivareal.com.br/venda/brasil/casa_residencial/" property="og:url"/> 5
  6
<meta content="" property="og:type"/> 7
  8
<meta content="" property="og:image"/> 9
  10
<meta content="image/jpeg" property="og:image:type"/> 11
  12
<meta content="" property="og:image:secure_url"/> 13
  14
<meta content="Casas à venda no Brasil - Viva Real" property="og:title"/> 15
  16
<meta content="Mais de 1.097.694 casas para comprar no Brasil. Seu imóvel ideal está aqui no Viva Real! Acesse agora as melhores ofertas de casas." property="og:description"/> 17
  18
<meta content="Mais de 1.097.694 casas para comprar no Brasil. Seu imóvel ideal está aqui no Viva Real! Acesse agora as melhores ofertas de casas." name="description"/> 19
  20
<title> Casas à venda no Brasil - Viva Real </title> 21
  22
<link href="https://www.vivareal.com.br/venda/bra

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-BR" prefix="og: https://ogp.me/ns#">
 <head>
  <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/>
  <meta content="omni2c019e0" name="omniverify"/>
  <meta content="https://www.vivareal.com.br/venda/brasil/casa_residencial/" property="og:url"/>
  <meta content="" property="og:type"/>
  <meta content="" property="og:image"/>
  <meta content="image/jpeg" property="og:image:type"/>
  <meta content="" property="og:image:secure_url"/>
  <meta content="Casas à venda no Brasil - Viva Real" property="og:title"/>
  <meta content="Mais de 1.097.694 casas para comprar no Brasil. Seu imóvel ideal está aqui no Viva Real! Acesse agora as melhores ofertas de casas." property="og:description"/>
  <meta content="Mais de 1.097.694 casas para comprar no Brasil. Seu imóvel ideal está aqui no Viva Real! Acesse agora as melhores ofertas de casas." name="description"/>
  <title>
   Casas à venda no Brasil - Viva Real
  </title>
  <link href="https://www.viv

In [44]:
soup.contents[-2].contents[3].prettify()

'<body>\n <nav class="js-site-menu" id="js-site-menu">\n  <div class="site-menu__wrapper js-site-menu-wrapper">\n   <div class="site-menu js-site-menu-panel">\n    <header class="site-menu__header">\n     <button class="site-menu__close js-site-menu-close">\n      Fechar\n     </button>\n    </header>\n    <ul class="site-menu__list">\n     <li>\n      <button class="site-header__login site-header__login--mobile js-login" title="Entrar">\n       Entrar\n      </button>\n     </li>\n     <li>\n      <a class="js-track" href="/venda/?itl_id=1000177&amp;itl_name=vivareal_-_link-header_comprar_to_vivareal_resultado-pesquisa">\n       Comprar\n      </a>\n     </li>\n     <li>\n      <a class="js-track" href="/aluguel/?itl_id=1000178&amp;itl_name=vivareal_-_link-header_alugar_to_vivareal_resultado-pesquisa">\n       Alugar\n      </a>\n     </li>\n     <li>\n      <a class="js-track" href="/imoveis-lancamento/?itl_id=1000179&amp;itl_name=vivareal_-_link-header_lancamentos_to_vivareal_result

In [47]:
len(soup.contents[-2].contents[3].contents)

32

In [72]:
from langchain.document_loaders import WebBaseLoader

urls = [url]
loader = WebBaseLoader(urls)
docs = loader.load()

# LLM Setting

In [50]:
content = soup.contents[-2].contents[3].contents

In [52]:
type(content[1])

bs4.element.Tag

In [54]:
str_content_list = []
for c in content:
    str_content_list.append(str(c))

In [83]:
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

In [88]:
docs = []
count = 0
for i in str_content_list:
    docs.append(Document(page_content=i, metadata = None))
    count += 1 

In [100]:
llm = Ollama(
    model='llama2:13b', callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

embeddings = OllamaEmbeddings(model='llama2:13b')

vector_db = Chroma.from_documents(docs, embedding=embeddings , persist_directory='.')
vector_db.persist()

ValueError: Error raised by inference API HTTP code: 500, {"error":"failed to generate embedding"}

In [98]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_db.as_retriever())


question = "writhe a code in python that extract information from the html, a webscraper that returns the the main information as number of rooms, bathroom and the total size of vivareal.com.br"
result = qa_chain({"query": question})

I am not able to write codes in python or perform web scraping tasks directly. However, I can provide a general approach for creating a web scraper in python that extracts information from the html of vivareal.com.br. Here's an outline:

1. Install necessary libraries: before writing any code, make sure you have installed required python packages like beautifulsoup4, requests, and re. These libraries will help us to parse html, make http requests, and perform regular expressions, respectively.

2. Make a request: use the requests library's get() function to fetch the html content from vivareal.com.br website. The url can be passed as an argument to this method.

3. Parse html: once we have fetched the webpage, parse it using beautifulsoup4's parser(). This will return a beautifulsoup object that can be used to extract data.

4. Navigate the dom: navigate through the dom using beautifulsoup's find() and find_all() methods to locate elements containing the required information (number of